In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
from torchsummary import summary

import numpy as np
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [10]:
'''
def clone_module_list(module: M, n: int) -> TypedModuleList[M]:
    """
    ## Clone Module

    Make a `nn.ModuleList` with clones of a given module
    """
    return TypedModuleList([copy.deepcopy(module) for _ in range(n)])
'''

'\ndef clone_module_list(module: M, n: int) -> TypedModuleList[M]:\n    """\n    ## Clone Module\n\n    Make a `nn.ModuleList` with clones of a given module\n    """\n    return TypedModuleList([copy.deepcopy(module) for _ in range(n)])\n'

In [71]:
class VisionTransformer(nn.Module):
    def __init__(self, batch_size, num_classes, dim, depth, heads, mlp_dim, img_dim = [3,224,224], patch_dim = [3,56,56], dim_head = 64):
        super().__init__()
        image_h = img_dim[1]
        image_w = img_dim[2]
        patch_h = patch_dim[1]
        patch_w = patch_dim[2]

        n_patches = (image_h // patch_h) * (image_w // patch_w)
        patch_dim = img_dim[0] * patch_h * patch_w

        self.to_patch_embedding = nn.Sequential(
            nn.Unfold(kernel_size=(patch_h, patch_w), stride=(patch_h, patch_w))
        )

    def forward(self, img):
        x = self.to_patch_embedding(img)

        return x

In [72]:
model = VisionTransformer(batch_size=32, num_classes=10, dim=64, depth=8, heads=64, mlp_dim=256)

In [81]:
x = torch.randn([16,3,224,224])    #(b,c,h,w)
y = model(x)

In [82]:
print(x.shape)
print(y.shape)

torch.Size([16, 3, 224, 224])
torch.Size([16, 9408, 16])
